In [2]:
import numpy as np
import pandas as pd
import re
import time as t
import sys

import nltk
from nltk.corpus import stopwords
nltk.download
from nltk.tokenize import word_tokenize

In [3]:
data = pd.read_csv("C:/dong/basic/data/netflix_titles.csv")
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [4]:
# e
print(data.shape)
print(data.isna().sum())
print(data["director"].unique)
len(data)

(7787, 12)
show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64
<bound method Series.unique of 0                     NaN
1       Jorge Michel Grau
2            Gilbert Chan
3             Shane Acker
4          Robert Luketic
              ...        
7782          Josef Fares
7783          Mozez Singh
7784                  NaN
7785                  NaN
7786             Sam Dunn
Name: director, Length: 7787, dtype: object>


7787

In [5]:
data2 = data.fillna(-1)
print(data2.isna().sum())
print(data.isna().sum())

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64
show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64


In [30]:
a1 = data.index[data['director'].isnull()]
a2 = data.index[data['cast'].isnull()]
a3 = data.index[data['country'].isnull()]
a4 = data.index[data['date_added'].isnull()]
a5 = data.index[data['rating'].isnull()]
a= [a1, a2, a3, a4, a5]

In [24]:
data.loc[0,:]

show_id                                                        s1
type                                                      TV Show
title                                                          3%
director                                                      NaN
cast            João Miguel, Bianca Comparato, Michel Gomes, R...
country                                                    Brazil
date_added                                        August 14, 2020
release_year                                                 2020
rating                                                      TV-MA
duration                                                4 Seasons
listed_in       International TV Shows, TV Dramas, TV Sci-Fi &...
description     In a future where the elite inhabit an island ...
Name: 0, dtype: object

In [34]:
b1,b2,b3,b4,b5 = 0,0,0,0,0
b= [b1,b2,b3,b4,b5]
for i in range(5):
    b[i] = data.loc[a[i],:]
    print(b[i].shape)
    print("------")
    print(b[i].head(3))
    print("======================")

(2389, 12)
------
   show_id     type   title director  \
0       s1  TV Show      3%      NaN   
11     s12  TV Show    1983      NaN   
16     s17  TV Show  Feb-09      NaN   

                                                 cast                country  \
0   João Miguel, Bianca Comparato, Michel Gomes, R...                 Brazil   
11  Robert Więckiewicz, Maciej Musiał, Michalina O...  Poland, United States   
16  Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...                    NaN   

           date_added  release_year rating   duration  \
0     August 14, 2020          2020  TV-MA  4 Seasons   
11  November 30, 2018          2018  TV-MA   1 Season   
16     March 20, 2019          2018  TV-14   1 Season   

                                            listed_in  \
0   International TV Shows, TV Dramas, TV Sci-Fi &...   
11  Crime TV Shows, International TV Shows, TV Dramas   
16                  International TV Shows, TV Dramas   

                                          

In [ ]:
data2 = data.dropna(subset=['cast', 'country', 'rating'])
data2.shape

In [ ]:
movies = data[data['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
movies.head()

In [ ]:
actors = []

for i in movies['cast']:
    actor = re.split(r', \s*', i)
    actors.append(actor)
    
flat_list = []
for sublist in actors:
    for item in sublist:
        flat_list.append(item)
        
actors_list = sorted(set(flat_list))

binary_actors = [[0] * 0 for i in range(len(set(flat_list)))]

for i in movies['cast']:
    k = 0
    for j in actors_list:
        if j in i:
            binary_actors[k].append(1.0)
        else:
            binary_actors[k].append(0.0)
        k+=1
        
binary_actors = pd.DataFrame(binary_actors).transpose()
        
directors = []

for i in movies['director']:
    if pd.notna(i):
        director = re.split(r', \s*', i)
        directors.append(director)
    
flat_list2 = []
for sublist in directors:
    for item in sublist:
        flat_list2.append(item)
        
directors_list = sorted(set(flat_list2))

binary_directors = [[0] * 0 for i in range(len(set(flat_list2)))]

for i in movies['director']:
    k = 0
    for j in directors_list:
        if pd.notna(j):
            binary_directors[k].append(0.0)
        elif j in i:
            binary_directors[k].append(1.0)
        else:
            binary_directors[k].append(0.0)
        k+=1
        
binary_directors = pd.DataFrame(binary_directors).transpose()
        
countries = []

for i in movies['country']:
    country = re.split(r', \s*', i)
    countries.append(country)
    
flat_list3 = []
for sublist in countries:
    for item in sublist:
        flat_list3.append(item)
        
countries_list = sorted(set(flat_list3))

binary_countries = [[0] * 0 for i in range(len(set(flat_list3)))]

for i in movies['country']:
    k = 0
    for j in countries_list:
        if j in i:
            binary_countries[k].append(1.0)
        else:
            binary_countries[k].append(0.0)
        k+=1
        
binary_countries = pd.DataFrame(binary_countries).transpose()

genres = []

for i in movies['listed_in']:
    genre = re.split(r', \s*', i)
    genres.append(genre)
    
flat_list4 = []
for sublist in genres:
    for item in sublist:
        flat_list4.append(item)
        
genres_list = sorted(set(flat_list4))

binary_genres = [[0] * 0 for i in range(len(set(flat_list4)))]

for i in movies['listed_in']:
    k = 0
    for j in genres_list:
        if j in i:
            binary_genres[k].append(1.0)
        else:
            binary_genres[k].append(0.0)
        k+=1
        
binary_genres = pd.DataFrame(binary_genres).transpose()

ratings = []

for i in movies['rating']:
    ratings.append(i)

ratings_list = sorted(set(ratings))

binary_ratings = [[0] * 0 for i in range(len(set(ratings_list)))]

for i in movies['rating']:
    k = 0
    for j in ratings_list:
        if j in i:
            binary_ratings[k].append(1.0)
        else:
            binary_ratings[k].append(0.0)
        k+=1
        
binary_ratings = pd.DataFrame(binary_ratings).transpose()